In [1]:
import os
import torch
from PIL import Image
import pandas as pd
import matplotlib.pyplot as plt
import open_clip
from tqdm import tqdm
import numpy as np

In [2]:
# --- CONFIGURATION ---

# 1. INPUT: Path to your JSON file containing image paths and their corresponding prompts.
JSON_INPUT_FILE = "clip.json" 

# 2. OUTPUT: The main directory where all results will be saved.
#    Subfolders for each experiment will be created automatically inside this directory.
MAIN_OUTPUT_DIR = "./Output/CLIP_Ranking_Results"

# 3. MODEL: The OpenCLIP model to use for evaluation.
MODEL_NAME = 'ViT-B-32'
PRETRAINED_DATASET = 'laion2b_s34b_b79k'

# 4. THRESHOLD: How much above the average score an image must be to be considered "good".
#    For example, 1.05 means 5% above the average.
THRESHOLD_MULTIPLIER = 1.05 

# 5. NEGATIVE PROMPT: A general negative prompt to apply to all experiments.
NEGATIVE_PROMPT = "bad anatomy, bad hands, blurry, low resolution, worst quality, jpeg artifacts, ugly, deformed, disfigured, text, watermark, signature, cartoon, 3d, cgi, anime"


In [3]:
def load_model():
    """Loads the OpenCLIP model, preprocessor, and tokenizer."""
    print(f"Loading model: {MODEL_NAME} pretrained on {PRETRAINED_DATASET}...")
    model, _, preprocess = open_clip.create_model_and_transforms(MODEL_NAME, pretrained=PRETRAINED_DATASET)
    tokenizer = open_clip.get_tokenizer(MODEL_NAME)
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model = model.to(device).eval()
    print(f"Model loaded successfully on device: {device}")
    return model, preprocess, tokenizer, device

def group_data_by_experiment(json_path):
    """Parses the JSON file and groups image paths and prompts by experiment name."""
    print(f"Grouping data from {json_path}...")
    try:
        df = pd.read_json(json_path)
    except Exception as e:
        print(f"Error reading JSON file: {e}")
        return None

    experiments = {}
    for _, row in df.iterrows():
        path = row['image_path']
        prompt = row['prompt']
        try:
            exp_name = path.split(os.sep)[1]
        except IndexError:
            print(f"Warning: Could not determine experiment name for path '{path}'. Skipping.")
            continue
            
        if exp_name not in experiments:
            experiments[exp_name] = {'prompt': prompt, 'images': []}
        experiments[exp_name]['images'].append(path)
        
    print(f"Found {len(experiments)} experiments to process.")
    return experiments

def process_experiment(exp_name, data, model, preprocess, tokenizer, device, neg_embed, output_dir_base):
    """Processes a single experiment: scores images, ranks them, and saves all outputs."""
    print(f"\n--- Starting Experiment: {exp_name} ---")
    
    positive_prompt = data['prompt']
    image_paths = data['images']
    
    exp_output_dir = os.path.join(output_dir_base, exp_name)
    good_images_dir = os.path.join(exp_output_dir, "good_images")
    os.makedirs(good_images_dir, exist_ok=True)
    
    with torch.no_grad():
        pos_tokens = tokenizer([positive_prompt]).to(device)
        pos_embed = model.encode_text(pos_tokens).float()
        pos_embed /= pos_embed.norm(dim=-1, keepdim=True)

    results = []
    for path in tqdm(image_paths, desc=f"Scoring images for {exp_name}"):
        try:
            image = preprocess(Image.open(path).convert("RGB")).unsqueeze(0).to(device)
            with torch.no_grad():
                img_embed = model.encode_image(image).float()
                img_embed /= img_embed.norm(dim=-1, keepdim=True)
                pos_score = (img_embed @ pos_embed.T).item()
                neg_score = (img_embed @ neg_embed.T).item()
                final_score = pos_score - neg_score
                results.append((os.path.basename(path), final_score, pos_score, neg_score))
        except FileNotFoundError:
            print(f"Warning: File not found, skipping: {path}")
        except Exception as e:
            print(f"Error processing {os.path.basename(path)}: {e}")

    if not results:
        print(f"No images were successfully processed for experiment {exp_name}. Skipping.")
        return

    df = pd.DataFrame(results, columns=["Image", "Score", "Positive", "Negative"])
    df.sort_values(by="Score", ascending=False, inplace=True)
    df.insert(0, "Rank", range(1, len(df) + 1))
    
    average_score = df["Score"].mean()
    good_threshold = average_score * THRESHOLD_MULTIPLIER
    print(f"Average Score: {average_score:.4f} | Dynamic Good Threshold: {good_threshold:.4f}")

    csv_path = os.path.join(exp_output_dir, f"ranked_results_{exp_name}.csv")
    df.to_csv(csv_path, index=False)
    print(f"Ranked CSV saved to: {csv_path}")

    plt.style.use('seaborn-v0_8-whitegrid')
    plt.figure(figsize=(12, 7))
    plt.plot(df["Rank"], df["Score"], marker='o', linestyle='-', color='b', label='Total Score (Pos - Neg)')
    plt.axhline(y=good_threshold, color='g', linestyle='--', label=f'Good Threshold ({good_threshold:.2f})')
    plt.axhline(y=average_score, color='r', linestyle=':', label=f'Average Score ({average_score:.2f})')
    plt.title(f"CLIP Score Ranking for: {exp_name}", fontsize=16)
    plt.xlabel("Rank", fontsize=12)
    plt.ylabel("Score", fontsize=12)
    plt.legend()
    plt.tight_layout()
    plot_path = os.path.join(exp_output_dir, f"rank_plot_{exp_name}.png")
    plt.savefig(plot_path)
    plt.close()
    print(f"Plot saved to: {plot_path}")

    good_images_df = df[df["Score"] >= good_threshold]
    print(f"Found {len(good_images_df)} images meeting the 'good' threshold.")
    for _, row in good_images_df.iterrows():
        original_path = next((p for p in image_paths if os.path.basename(p) == row["Image"]), None)
        if original_path:
            try:
                dest_path = os.path.join(good_images_dir, row["Image"])
                Image.open(original_path).save(dest_path)
            except Exception as e:
                print(f"Failed to copy {row['Image']}: {e}")
    print(f"Good images saved to: {good_images_dir}")
    
print("✅ Helper functions defined.")

✅ Helper functions defined.


In [4]:
# Load the model and tokenizer
model, preprocess, tokenizer, device = load_model()

# Group the input images and prompts from the JSON file
experiments = group_data_by_experiment(JSON_INPUT_FILE)

# Pre-encode the global negative prompt once for efficiency
if experiments:
    with torch.no_grad():
        neg_tokens = tokenizer([NEGATIVE_PROMPT]).to(device)
        neg_embed = model.encode_text(neg_tokens).float()
        neg_embed /= neg_embed.norm(dim=-1, keepdim=True)
    print("✅ Initial setup complete. Negative prompt is encoded.")
else:
    print("❌ No experiments found or error in loading data. Cannot proceed.")

Loading model: ViT-B-32 pretrained on laion2b_s34b_b79k...


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


open_clip_pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

KeyboardInterrupt: 

In [ ]:
if experiments:
    # Process each experiment sequentially
    for exp_name, data in experiments.items():
        process_experiment(exp_name, data, model, preprocess, tokenizer, device, neg_embed, MAIN_OUTPUT_DIR)
        
    print("\n✅ All experiments processed successfully!")
else:
    print("❌ Cannot run experiments because data was not loaded correctly.")